In [ ]:
#bibliotecas padrao
import pandas as pd #csv, dataframes

#preparacao dos dados
import re #expressoes regulares
from sklearn.feature_extraction.text import CountVectorizer #vetorizacao via BoW

#modelo
from sklearn.naive_bayes import MultinomialNB #modelo tipo navie bayes

<h1>Sumário</h1>

* Importando e limpando os dados<br> 
* Preparação dos dados<br>
 * RegEx<br>
* Criando o modelo<br>
 * Vetorização<br> 
   * BoW<br>
   * TD - IDF<br>
* Treinamento<br>
* Teste<br>
* Predições<br>
* Respostas

<h1>Importando e limpando os dados</h1>

* <p1>Os <a href = "https://www.kaggle.com/thiagopanini/e-commerce-sentiment-analysis-eda-viz-nlp">dados</a> referem-se ao e-commerce no território brasileiro entre 09/2016 até 10/2018.</p1><br><br>
* <p1>O arquivo de interesse chama-se 'olist_order_reviews_dataset.csv' e contém informações sobre os reviews relacionados às ordens de compra no e-commerce no Brasil no mesmo intervalo temporal.</p1><br><br>
* <p1> Os comentarios dos reviews estão na coluna 'review_comment_message', e as notas estão na coluna 'review_score'.</p1><br><br>

In [ ]:
#importando os dados de interesse
df = pd.read_csv('/kaggle/input/brazilian-ecommerce/olist_order_reviews_dataset.csv', 
                 usecols = ['review_score','review_comment_message'])

#renomeando colunas
df.columns = ['score','comentarios']

In [ ]:
#removendo dados ausentes do dataframe
df.dropna(inplace = True)

#resetando índice
df.reset_index(inplace = True, drop = True)

#imprime tuple(#linhas,#colunas)
df.shape

<h1>Preparação dos dados</h1>

<p1>Segundo <a href = "https://www.kaggle.com/thiagopanini/e-commerce-sentiment-analysis-eda-viz-nlp"> Panini</a>, para fazer a análise sentimental é necessário preparar os dados e transformá-los em vetores, para que por fim sejam interpretados por um modelo de Machine Learning. A preparação consiste, dentre outros passos, em aplicar Expressões Regulares (RegEx) no conjunto de dados, afim de identificar padrões e extraí-los. A vetorização depende do modelo utilizado, e consiste em representar palavras vetorialmente em função da sua frequência em um documento e/ou de sua importância.

<h2>RegEx</h2>

* É preciso criar uma função que substitua as quebras de linhas encontradas nos textos por espaços em branco
* É preciso criar uma função que substitua os valores numéricos pela palavra 'numero'
* É preciso criar uma função que substitua caracteres especiais por espaços em branco

In [ ]:
def reQuebraDeLinha(texto_lista):
    #quebras de linha - \n\r2
    #re.sub -> expressão regular que substitui um padrão por outro
    
    texto_lista = [re.sub('\r\n',' ', i) for i in texto_lista]   # list comprehension para substituir
                                                                      # as quebras de linha por espaços em branco
    return texto_lista

In [ ]:
def numTransform(texto_lista):
    #[] indica quais caracteres estou considerando
    #0-9 -> numeros no intervalo (0,9)
    #'+' -> na documentação diz que significa que o que estiver imediatamente antes dele precisa aparecer 1 ou mais vezes,
    #       porém não entendi muito bem. Mas sem o + na RegEx a palavra 'numero' se repete
    
    texto_lista = [re.sub('[0-9]+', 'numero ', i) for i in texto_lista]
    return texto_lista

In [ ]:
'''
def negacao(texto_lista):
    #[] indica quais caracteres estou considerando
    texto_lista = [re.sub('([nN][ãÃaA][oO]|[ñÑ]| [nN] )', 'negacao', i) for i in texto_lista]
    return texto_lista
'''

In [ ]:
def charEsp(texto_lista):
    #\W caracter nao alfa-numerico (^[a-zA-Z0-9])
    texto_lista = [re.sub('\W', ' ', i) for i in texto_lista]
    return texto_lista

In [ ]:
def preparacao(texto_lista):
    #aplicando a funcao requebraDeLinha()
    texto_lista = reQuebraDeLinha(texto_lista)

    #aplicando a função numTransform()
    texto_lista = numTransform(texto_lista)

    #aplicando a função negacao()
    #texto_lista = negacao(texto_lista)
    
    #aplicando a função charEsp()
    texto_lista = charEsp(texto_lista)
    
    return texto_lista

In [ ]:
df.comentarios = preparacao(df['comentarios'])

In [ ]:
# 0 = negativo e 1 = positivo
mapa_score = {1: 'negativo', 2: 'negativo', 3: 'negativo', 4: 'positivo', 5: 'positivo'}
df['sentimento'] = df['score'].map(mapa_score)
df.head()

In [ ]:
df.sentimento.value_counts()

<h1>Criando o modelo</h1>

<h2>Vetorização</h2>

<h3>Bag of Words (BoW)</h3>

Segundo <a href = "https://www.analyticsvidhya.com/blog/2020/02/quick-introduction-bag-of-words-bow-tf-idf/">Analytics Vidhya</a>, o modelo BoW é definido como uma forma simples de fazer uma representação vetorial de um dado termo "t" presente em um dado documento "d", a partir de sua frequência de ocorrência no mesmo. O vetor é gerado de forma tal, que para cada termo presente no texto está associado um valor binário (0 ou 1), onde 0 significa não ocorrência e 1 indica ocorrência do termo em um dado documento.

<h3>Term Frequency Inverse Document Frequency (TF - IDF)</h3>

O modelo TF-IDF consiste em quantificar a importância de cada palavra no conjunto de dados e representá-las como um vetor, onde cada elemento destes é um número do tipo float que está associado a frequência e ao peso de cada termo no documento, e quanto maior este número, mais relevante é a palavra.</p1><br>

Segundo <a href = "https://www.analyticsvidhya.com/blog/2020/02/quick-introduction-bag-of-words-bow-tf-idf/">Analytics Vidhya</a>, o TF-IDF score é dado por: <br><br>

$(TF - IDF)_{t,d} = TF_{t,d} * IDF_{t,d}$<br><br>
<p1> onde "t" e "d" representam o termo analisado em um dado documento, respectivamente. Por definição, também temos a frequência do termo "t" no documento "d", dada por:</p1><br>
* $TF_{t,d} = \frac{\textrm{nº de vezes que o termo "t" aparece no documento "d"}}{\textrm{número total de termos no documento "d"}}$<br><br>

E a importância do termo "t", definida como:<br>

* $IDF_{t,d} = \Big(\frac{\textrm{número de documentos}}{\textrm{número de documentos que contém o termo "t"}}\Big)$<br><br>

Ainda segundo <a href = "https://www.analyticsvidhya.com/blog/2020/02/quick-introduction-bag-of-words-bow-tf-idf/">Analytics Vidhya</a>, apesar do modelo BoW ser mais simples no que se refere a interpretação, o modelo TD-IDF possui um desempenho maior quando comparado com o modelo Bag of Words em modelos de Machine Learning, uma vez que o TD-IDF traz informações sobre a importância das palavras e o modelo Bag of Words apenas traz informações sobre a ocorrência destas no documento. Neste notebook, por questões de praticidade usarei o modelo BoW

In [ ]:
#instanciando o vetor
vectorizer = CountVectorizer(max_features = 300, min_df = 7, max_df = 0.8)

#vetorizando
comentarios_vectorized = vectorizer.fit_transform(df['comentarios'])

print(comentarios_vectorized.shape)
#print(comentarios_vectorized.A) #vizualização da matriz

<h1>Treinamento</h1>

In [ ]:
#treinando o modelo
modelo = MultinomialNB() #instanciando o modelo de ML navie bayes-multinomial
modelo.fit(comentarios_vectorized, df['sentimento'])

In [ ]:
'''
for comentario, classificacao in zip(df['comentarios'], modelo.predict(comentarios_vectorized)):
    result_coment.append(comentario)
    result_class.append(classificacao)
    print(comentario + '  ' + classificacao)
'''

<h1>Predições</h1>

In [ ]:
predicao = modelo.predict(comentarios_vectorized) #predicoes feitas pelo modelo
proba = modelo.predict_proba(comentarios_vectorized) #probabilidade de cada classe

In [ ]:
result = pd.DataFrame(columns = ['predicao', 'comentarios'])
result.predicao = predicao
result.comentarios = df['comentarios']
result['predicao'].value_counts()

<h1>Respostas</h1>